# Metric Analysis for ECON model

In [19]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
import torchinfo
import os
import sys

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../../workspace/models/econ/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from models.econ.code.q_autoencoder import AutoEncoder
from models.econ.code.autoencoder_datamodule import AutoEncoderDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from common.metrics.gradient import Gradient



Select the desired ECON model:

In [20]:
base_path = "/loss_landscape/checkpoint/different_knobs_subset_10/"
batch_size = 64
learning_rate = 0.1
precision = 2
size = 'small'

In [21]:
def load_model(path, batch_size, learning_rate, precision, size):
    model_path = base_path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/net_1_best.pkl'
    model = AutoEncoder(
        quantize=(precision < 32),
        precision=[
            precision,
            precision,
            precision+3
        ],
        learning_rate=learning_rate,
        econ_type=size
    )
    model(torch.randn((1, 1, 8, 8)))  # Update tensor shapes 
    print(model_path)
    model_param = torch.load(model_path)
    model.load_state_dict(model_param['state_dict'])
    return model


model = load_model(base_path, batch_size, learning_rate, precision, size)
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)            

/loss_landscape/checkpoint/different_knobs_subset_10/bs64_lr0.1/ECON_2b/small/net_1_best.pkl


Layer (type:depth-idx)                   Output Shape              Param #
AutoEncoder                              [1, 1, 8, 8]              --
├─QuantizedEncoder: 1-1                  [1, 16]                   --
│    └─QuantAct: 2-1                     [1, 1, 8, 8]              --
│    └─QuantConv2d: 2-2                  [1, 1, 4, 4]              20
│    └─ReLU: 2-3                         [1, 1, 4, 4]              --
│    └─QuantAct: 2-4                     [1, 1, 4, 4]              --
│    └─Flatten: 2-5                      [1, 16]                   --
│    └─QuantLinear: 2-6                  [1, 16]                   272
│    └─ReLU: 2-7                         [1, 16]                   --
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Linear: 2-8                       [1, 128]                  2,176
│    └─ReLU: 2-9                         [1, 128]                  --
│    └─Unflatten: 2-10                   [1, 8, 4, 4]              --
│    └─Conv

Setup the data loader:

In [24]:
data_path = '../../data/ECON/Elegun'
processed_file = 'nELinks5.npy'

data_module = AutoEncoderDataModule(
    data_dir=data_path,
    data_file=os.path.join(data_path, processed_file),
    batch_size=batch_size,
    num_workers=4
)
# checek if we have processed the data
if not os.path.exists(os.path.join(data_path, processed_file)):
    print('Processing the data...')
    data_module.process_data(save=True)

data_module.setup(0)

data_loader = data_module.val_dataloader()

Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32


## Gradient computation

In [25]:
metric = Gradient(
    model=model,
    data_loader=data_loader,
    loss=model.loss
)

metric.compute()

Computing the gradients...


ValueError: too many values to unpack (expected 2)